In [ ]:
%pip install shap

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
import pandas as pd
import numpy as np
import shap
import os
import json
import joblib

In [ ]:
def evaluate_classification(y_true, y_pred, results, solver_name, label):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"{label} | Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

    results.append({
        "Solver": solver_name,
        "Dataset": label,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1
    })

In [ ]:
import os
import numpy as np
import pandas as pd
import shap

def log_shap_and_importance_classification(model, X_val, y_val, pred_val, features, solver_name, target):

    background = shap.sample(X_val, min(100, X_val.shape[0]), random_state=42)

    try:
        explainer   = shap.KernelExplainer(model.predict_proba, background)
        shap_values = explainer.shap_values(X_val, nsamples=100)

        # normalize into (n_samples, n_features)
        arrs = shap_values if isinstance(shap_values, list) else [shap_values]
        proc = []
        for a in arrs:
            a = np.array(a)
            if a.ndim == 4:
                a = np.squeeze(a)
            if a.ndim == 3:
                a = a.mean(axis=-1)
            
            if a.ndim != 2:
                raise ValueError(f"Error shap array shape {a.shape}")
            proc.append(a)

        # stack to (n_samples, n_features, n_classes)
        arr = np.stack(proc, axis=-1)

       
        feat_imp = np.mean(np.abs(arr), axis=(0,2))
        assert feat_imp.shape[0] == len(features)
        shap_row = feat_imp[np.newaxis, :]

   
        shap_df = pd.DataFrame(shap_row, columns=features)
        shap_df["target"] = target
        shap_df["solver"] = solver_name
        out_dir = "./svm_class/svm_shap_values"
        os.makedirs(out_dir, exist_ok=True)
        shap_df.to_csv(
            f"{out_dir}/shap_{solver_name}_{target}_classification.csv",
            index=False
        )

       
        if hasattr(model, "coef_"):
            coefs = np.abs(model.coef_)
            coef_imp = coefs.mean(axis=0) if coefs.ndim == 2 else coefs.flatten()
        else:
            coef_imp = np.zeros(len(features))

        imp_df = pd.DataFrame({
            "feature": features,
            "model_importance": coef_imp,
            "target": target,
            "solver": solver_name
        })
        imp_dir = "./svm_class/svm_feature_importance"
        os.makedirs(imp_dir, exist_ok=True)
        imp_df.to_csv(
            f"{imp_dir}/svm_feature_importance_classification.csv",
            mode='a', index=False,
            header=not os.path.exists(f"{imp_dir}/svm_feature_importance_classification.csv")
        )

        # Top-5
        top5 = imp_df.nlargest(5, "model_importance")
        top5.to_csv(
            f"{imp_dir}/svm_top5_feature_importance_classification.csv",
            mode='a', index=False,
            header=not os.path.exists(f"{imp_dir}/svm_top5_feature_importance_classification.csv")
        )

        print(f"SHAP saved for {solver_name}-{target}.")

    except Exception as e:
        print(f"SHAP failed for {solver_name}-{target}: {e}")

In [ ]:
def train_svm_classifier_for_solver(solver_name, train_file, test_file, val_file):
    print(f"\nSolver: {solver_name}")

    df_train = pd.read_csv(train_file).dropna()
    df_test  = pd.read_csv(test_file).dropna()
    df_val   = pd.read_csv(val_file).dropna()

    features = [
        "number_of_elements","capacity","max_weight","min_weight","mean_weight",
        "median_weight","std_weight","weight_range","max_profit","min_profit","mean_profit",
        "median_profit","std_profit","profit_range","renting_ratio","mean_weight_profit_ratio",
        "median_weight_profit_ratio","capacity_mean_weight_ratio","capacity_median_weight_ratio",
        "capacity_std_weight_ratio","std_weight_profit_ratio","weight_profit_correlation",
        "ram","cpu_cores"
    ]
    target_cols = ["solution_time", "optimality_gap", "peak_memory"]

 
    bins_dir = os.path.join(BINS_BASE_DIR, f"{solver_name}_bins")
    if not os.path.isdir(bins_dir):
        raise FileNotFoundError(f"No bins directory: {bins_dir}")
    json_files = [f for f in os.listdir(bins_dir) if f.endswith("_bins.json")]
    if len(json_files) != 1:
        raise FileNotFoundError(f"Expected one json in {bins_dir}, found: {json_files}")
    bin_path = os.path.join(bins_dir, json_files[0])
    with open(bin_path, "r") as f:
        bin_edges_dict = json.load(f)

    
    scaler = StandardScaler().fit(df_train[features])
    X_train = scaler.transform(df_train[features])
    X_test  = scaler.transform(df_test[features])
    X_val   = scaler.transform(df_val[features])


    base_out = "./svm_class"
    os.makedirs(f"{base_out}/svm_tuning", exist_ok=True)
    os.makedirs(f"{base_out}/svm_configs", exist_ok=True)
    os.makedirs(f"{base_out}/svm_classifier_models", exist_ok=True)

    results = []
    for target in target_cols:
        if target not in bin_edges_dict:
            print(f" No bins for '{target}'.")
            continue
        edges = bin_edges_dict[target]

        
        def to_bins(arr, edges):
            labels = np.digitize(arr, edges[:-1], right=False) - 1
            return np.clip(labels, 0, len(edges) - 2)

        y_train = to_bins(df_train[target].values, edges)
        y_test  = to_bins(df_test [target].values, edges)
        y_val   = to_bins(df_val  [target].values, edges)

       
        max_train = y_train.max()
        y_test = np.clip(y_test, 0, max_train)
        y_val  = np.clip(y_val,   0, max_train)

        #  Skip if only one class in train
        cls_train = np.unique(y_train)
        if len(cls_train) < 2:
            print(f"Skipping '{target}': only one class {cls_train}")
            continue

        # Hyperparameter grid search
        param_grid = {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"]
        }
        best_f1     = -np.inf
        tuning_logs = []
        best_model  = None

        for params in ParameterGrid(param_grid):
            model = SVC(**params, probability=True, random_state=42)
            model.fit(X_train, y_train)
            pred_val = model.predict(X_val)
            f1 = f1_score(y_val, pred_val, average='weighted')
            tuning_logs.append({**params, "f1_score": f1})

            if f1 > best_f1:
                best_f1       = f1
                best_model    = model
                best_params   = params
                best_y_test    = y_test
                best_y_val     = y_val
                best_pred_test = model.predict(X_test)
                best_pred_val  = pred_val

        if best_model is None:
            print(f"No valid found '{target}'")
            continue

        # Save tuning logs & config
        pd.DataFrame(tuning_logs).to_csv(
            f"{base_out}/svm_tuning/tuning_svm_{solver_name}_{target}.csv", index=False
        ) 
        with open(f"{base_out}/svm_configs/best_svm_{solver_name}_{target}.json", "w") as f:
            json.dump(best_params, f, indent=4)

        # Evaluate 
        print(f"[{target} Test]")
        evaluate_classification(best_y_test,    best_pred_test, results,
                                solver_name, target)
        print(f"[{target} Val]")
        evaluate_classification(best_y_val,     best_pred_val,  results,
                                solver_name, target)

      
        model_path = f"{base_out}/svm_classifier_models/svm_{solver_name}_{target}.joblib"
        joblib.dump(best_model, model_path)
        print(f"Saved model{model_path}")

        
        log_shap_and_importance_classification(
            best_model, X_val, best_y_val, best_pred_val,
            features, solver_name, target
        )

  
    pd.DataFrame(results).to_csv(
        f"{base_out}/svm_evaluation_results_classification.csv",
        mode='a', index=False,
        header=not os.path.exists(f"{base_out}/svm_evaluation_results_classification.csv")
    )

In [ ]:
def run_all_models(base_folder):
    for root, dirs, files in os.walk(base_folder):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            csv_files = os.listdir(folder_path)

            train_file = [f for f in csv_files if f.endswith("_train.csv")]
            test_file = [f for f in csv_files if f.endswith("_test.csv")]
            val_file = [f for f in csv_files if f.endswith("_val.csv")]

            if train_file and test_file and val_file:
                train_fp = os.path.join(folder_path, train_file[0])
                test_fp = os.path.join(folder_path, test_file[0])
                val_fp = os.path.join(folder_path, val_file[0])

                solver_name = folder  
                train_svm_classifier_for_solver(solver_name, train_fp, test_fp, val_fp)

In [ ]:
base_folder = "Dataset/maximization/training_data/algorithms" #Path to training data
BINS_BASE_DIR = "Dataset/maximization/training_data/bins" #Path to bin data
run_all_models(base_folder)